## 处理所有现金流数据并导出到SWCFData中

In [1]:
import sys
sys.path.append("..")
from roeProcessTools import *
from cfProcessTools import *

In [2]:
dirPath = "/Users/zch/Desktop/internship/SWCFData/raw/"
cleanPath = "/Users/zch/Desktop/internship/SWCFData/clean/"
markPath = "/Users/zch/Desktop/internship/SWCFData/mark/"

In [3]:
import os

In [20]:
with os.scandir(dirPath) as it:
    for entry in it:
        if not entry.name.endswith("xlsx"):
            continue
        df = pd.read_excel(entry.path)
        df = cleanCFData(df, continuousBool=False)  # 一定要选False
        df.to_excel(cleanPath+entry.name, index=False)

In [21]:
dfs = {}
with os.scandir(cleanPath) as it:
    for entry in it:
        if not entry.name.endswith("xlsx"):
            continue
        df = pd.read_excel(entry.path)
        dfs[entry.name[:-5]] = df

In [31]:
marked = {}
for key, item in dfs.items():
    marked[key] = mark(item, n=8, center=False, gaussian=True, order=4, threshold=0.8, highPct=75, increaseBool=True)

In [26]:
# 导出selected
for key, item in marked.items():
    item.to_excel(markPath+key+".xlsx")

In [32]:
for key, item in marked.items():
    item["industry"] = key

In [40]:
temp = pd.concat(marked.values(), axis=0)

In [41]:
temp.reset_index(inplace=True, drop=True)

In [43]:
temp.to_excel("/Users/zch/Desktop/internship/SWCFData/marked.xlsx", index=False)

In [51]:
temp

,code,name,mark,startTime,industry
0,000088.SZ,盐田港,I,2017-09,交通运输
1,000099.SZ,中信海直,I,2019-06,交通运输
2,000755.SZ,山西路桥,I,2018-06,交通运输
3,000828.SZ,东莞控股,I,2017-09,交通运输
4,000885.SZ,城发环境,I,2017-03,交通运输
...,...,...,...,...,...
1129,601677.SH,明泰铝业,I,2019-03,有色金属
1130,601958.SH,金钼股份,I,2017-09,有色金属
1131,603003.SH,龙宇燃油,I,2018-09,有色金属
1132,603399.SH,吉翔股份,I,2018-09,有色金属


## cf和roe标记进行merge

In [44]:
roePath = "/Users/zch/Desktop/internship/SWROEData/selectedMark.xlsx"
roe = pd.read_excel(roePath)

In [47]:
merged = pd.merge(roe, temp, how="inner", on=["code", "name", "mark", "industry"], suffixes=("_roe", "_cf"))

In [49]:
merged.to_excel("cf和roe共同上升的公司.xlsx", index=False)

In [50]:
merged

,code,name,mark,startTime_roe,industry,startTime_cf
0,000799.SZ,酒鬼酒,I,2014-12,食品饮料,2020-03
1,002216.SZ,三全食品,I,2017-03,食品饮料,2019-12
2,002568.SZ,百润股份,I,2017-06,食品饮料,2017-09
3,002661.SZ,克明面业,I,2018-06,食品饮料,2018-06
4,300146.SZ,汤臣倍健,I,2017-03,食品饮料,2019-12
...,...,...,...,...,...,...
168,002683.SZ,宏大爆破,I,2016-12,化工,2016-12
169,002194.SZ,武汉凡谷,I,2018-06,通信,2018-12
170,002231.SZ,奥维通信,I,2019-09,通信,2019-09
171,002467.SZ,二六三,I,2018-03,通信,2020-03
